In [1]:


import SimpleITK as sitk
import numpy as np
import cv2
import os
import random
import shutil
from ultralytics import YOLO

import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import random
import pandas as pd



In [2]:
transform_default = transforms.Compose([transforms.ToTensor()])

In [3]:
def prepare_transforms():
    
    train_set = ImageFolder(root='SPIDER_training/train/', transform=transform_default)
    validate_set = ImageFolder(root='SPIDER_training/val/', transform=transform_default)
        
    data_loader = DataLoader(train_set, batch_size=32, num_workers=6, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_test = DataLoader(validate_set, batch_size=128, num_workers=2, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)

    return data_loader, data_loader_test

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=5):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 128, kernel_size=8, padding=1)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(128, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2,2)

        # flatten: 32 * 15 * 15
        self.linear1 = nn.Linear(32*15*15, 100)
        self.linear2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = torch.flatten(x,1)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x


class CNN2(nn.Module):
    def __init__(self, in_channels=3, num_classes=5):
        super(CNN2, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, 128, kernel_size=8, padding=1)
        self.pool1 = nn.MaxPool2d(2,2)
        
        self.conv2 = nn.Conv2d(128, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2,2)
        
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(2,2)
        
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(32*7*7, 100)
        self.linear2 = nn.Linear(100, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = F.relu(self.conv4(x))
        x = self.pool4(x)
        x = self.flatten(x)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [6]:


model = CNN()



In [7]:
model

CNN(
  (conv1): Conv2d(3, 128, kernel_size=(8, 8), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=7200, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=5, bias=True)
)

In [4]:


def test(model,data_loader_test, criterion,device):
    test_loss = 0
    correct = 0
    total = 0
    for i, (batch_X, batch_Y) in enumerate(data_loader_test):
        X = batch_X.to(device, non_blocking=True)
        Y = batch_Y.to(device, non_blocking=True)
        outputs = model(X)
        loss = criterion(outputs, Y)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == Y).sum().item()
        total += Y.size(0)
    avg_test_loss = test_loss / len(data_loader_test)
    test_accuracy = correct / total * 100
    print(f" Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")            
    return  avg_test_loss, test_accuracy



In [5]:
torch.cuda.is_available()

True

In [6]:
def model_train(lr, model, seed, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

    
    losses_tr = []
    accuracies_tr = []
    losses_val = []
    accuracies_val = []
    data_loader, data_loader_test = prepare_transforms()
            
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
        
    num_epochs = num_epochs
    
    prev_prev_prev_loss = float('inf')
    prev_prev_loss = float('inf')
    prev_loss = float('inf')
    curr_loss = float('inf')

    if isinstance(model, type):
        model_instance = model()
    else:
        model_instance = model
    model_instance.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_instance.parameters(), lr=lr)
    for epoch in range(num_epochs): 
        print('Entered the loop')
        model_instance.train()
        total_loss = 0
        correct = 0
        total = 0

        loop = tqdm(enumerate(data_loader), total=len(data_loader), ncols=100, colour='green')
        for i, (batch_X, batch_Y) in loop:
            X = batch_X.to(device, non_blocking=True)  
            Y = batch_Y.to(device, non_blocking=True)  
            optimizer.zero_grad()
            outputs = model_instance(X)
            loss = criterion(outputs, Y)
            loss.backward()
            optimizer.step()
                
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == Y).sum().item()
            total += Y.size(0)

            loop.set_description(f"Train Loss: {total_loss/(i+1):.4f}, Acc: {100*correct/total:.2f}%")
                
    
        avg_loss = total_loss / len(data_loader)
        train_accuracy = correct / total * 100
        train_losses.append(avg_loss)
        train_accuracies.append(train_accuracy)
        print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")
            
            
        model_instance.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            loop_val = tqdm(enumerate(data_loader_test), total=len(data_loader_test), ncols=100, colour='blue')
            for i, (batch_X, batch_Y) in loop_val:
                X = batch_X.to(device, non_blocking=True)
                Y = batch_Y.to(device, non_blocking=True)
                outputs = model_instance(X)
                loss = criterion(outputs, Y)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == Y).sum().item()
                total += Y.size(0)

                loop_val.set_description(f"Val Loss: {val_loss/(i+1):.4f}, Acc: {100*correct/total:.2f}%")
        avg_val_loss = val_loss / (140)
        val_accuracy = correct / total * 100
        print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")            
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)
    

            

    return model_instance,lr,val_losses, val_accuracies, train_losses, train_accuracies

In [15]:


lrate = [0.001, 0.0001]
models= [CNN2, CNN]
results = []
for lr in lrate:
    for model_class in models:  # np. CNN
        model, lr_ret, val_losses, val_accuracies, train_losses, train_accuracies = model_train(lr, model_class, 42, 20)
        
        # poprawna nazwa pliku
        model_filename = f"model_{model_class.__name__}_lr{lr_ret}.pt"
        torch.save(model.state_dict(), model_filename)
        
        results.append({
            "model_name": model_class.__name__,
            "learning_rate": lr_ret,
            "val_losses": val_losses,
            "val_accuracies": val_accuracies,
            "train_losses": train_losses,
            "train_accuracies": train_accuracies,
            "model_file": model_filename
        })


# zapis wszystkich wyników do CSV (jeśli nie potrzebujesz wszystkich list, np. tylko średnie)
df = pd.DataFrame(results)
df.to_csv("experiment_results_260116.csv", index=False)


Entered the loop


Train Loss: 1.2993, Acc: 41.56%: 100%|██████████████████████████████| 99/99 [01:46<00:00,  1.07s/it]


Epoch 1, Average Loss: 1.2993


Val Loss: 1.2383, Acc: 46.72%: 100%|██████████████████████████████████| 7/7 [00:22<00:00,  3.16s/it]


Epoch 1, Validation Loss: 0.0619, Accuracy: 46.72%
Entered the loop


...

Epoch 19, Validation Loss: 0.0868, Accuracy: 51.48%
Entered the loop


Train Loss: 0.3006, Acc: 89.23%: 100%|██████████████████████████████| 99/99 [00:06<00:00, 14.65it/s]


Epoch 20, Average Loss: 0.3006


Val Loss: 1.9735, Acc: 49.55%: 100%|██████████████████████████████████| 7/7 [00:00<00:00,  8.50it/s]


Epoch 20, Validation Loss: 0.0987, Accuracy: 49.55%


In [7]:
from torchvision.models import vit_b_16
model = vit_b_16( image_size=128)
model.heads.head = torch.nn.Linear(model.heads.head.in_features, 5)
model

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [8]:
lrate = [0.0001]
models= [model]
results = []
for lr in lrate:
    for model_class in models: 
        model, lr_ret, val_losses, val_accuracies, train_losses, train_accuracies = model_train(lr, model_class, 42, 75)
        
        # poprawna nazwa pliku
        model_filename = f"model_{model_class.__name__}_lr{lr_ret}.pt"
        torch.save(model.state_dict(), model_filename)
        
        results.append({
            "model_name": model_class.__name__,
            "learning_rate": lr_ret,
            "val_losses": val_losses,
            "val_accuracies": val_accuracies,
            "train_losses": train_losses,
            "train_accuracies": train_accuracies,
            "model_file": model_filename
        })


# zapis wszystkich wyników do CSV (jeśli nie potrzebujesz wszystkich list, np. tylko średnie)
df = pd.DataFrame(results)
df.to_csv("experiment_results_260116_vit.csv", index=False)

Entered the loop


Train Loss: 1.5066, Acc: 35.72%: 100%|██████████████████████████████| 99/99 [01:26<00:00,  1.14it/s]


Epoch 1, Average Loss: 1.5066


Val Loss: 1.2414, Acc: 48.78%: 100%|██████████████████████████████████| 7/7 [00:13<00:00,  1.97s/it]


Epoch 1, Validation Loss: 0.0621, Accuracy: 48.78%
Entered the loop


...

Train Loss: 0.0291, Acc: 99.11%: 100%|██████████████████████████████| 99/99 [01:27<00:00,  1.13it/s]


Epoch 75, Average Loss: 0.0291


Val Loss: 3.6001, Acc: 44.79%: 100%|██████████████████████████████████| 7/7 [00:08<00:00,  1.15s/it]


Epoch 75, Validation Loss: 0.1800, Accuracy: 44.79%


AttributeError: 'VisionTransformer' object has no attribute '__name__'

In [11]:
        model_filename = f"model_ViT_lr{lr_ret}.pt"
        torch.save(model.state_dict(), model_filename)
        
        results.append({
            "model_name": f"model_ViT_lr{lr_ret}",
            "learning_rate": lr_ret,
            "val_losses": val_losses,
            "val_accuracies": val_accuracies,
            "train_losses": train_losses,
            "train_accuracies": train_accuracies,
            "model_file": model_filename
        })


        # zapis wszystkich wyników do CSV (jeśli nie potrzebujesz wszystkich list, np. tylko średnie)
        df = pd.DataFrame(results)
        df.to_csv("experiment_results_260116_vit.csv", index=False)